In [18]:
import torch
import torchvision
from torchinfo import summary
import matplotlib.pyplot as plt
from torch import nn
from torchvision import transforms
from going_modular import data_setup, engine

torch.__version__, torchvision.__version__

('2.0.1+cu118', '0.15.2+cu118')

In [19]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [20]:
import os
import zipfile
from pathlib import Path
import requests

data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"
if image_path.is_dir():
    print(f"{image_path} directory exist , skipping...")
else:
    print("start downloading...")
    image_path.mkdir(parents=True, exist_ok=True)
    with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
        request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
        f.write(request.content)
    with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
        print("unzipping...")
        zip_ref.extractall(image_path)

    os.remove(data_path / "pizza_steak_sushi.zip")

data\pizza_steak_sushi directory exist , skipping...


In [21]:
train_dir = image_path / "train"
test_dir = image_path / "test"

In [22]:
from going_modular import data_setup

# Manual

In [23]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])
manual_transform = transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.ToTensor(),
    normalize
])

In [24]:
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir,test_dir, manual_transform, 32, 0)
train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x226f66ce790>,
 ['pizza', 'steak', 'sushi'])

In [25]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT

In [26]:
auto_transforms = weights.transforms()
auto_transforms

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [27]:
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir, test_dir, auto_transforms, 32, 0)
train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x226ff9edfd0>,
 ['pizza', 'steak', 'sushi'])

In [29]:
# old method in 0.13
# model = torchvision.models.efficientnet_b0(pretrained=True)
# 
# new in 0.15
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
model = torchvision.models.efficientnet_b0(weights=weights)
model

RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory